In [9]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


def recommend_books(favorites, n, cosine_sim=cosine_sim):

    # CSV 파일 불러오기
    df = pd.read_csv("../res/popular_books_ISBN3.csv")

    # 작가 이름이 완전히 같은지 체크
    df['same_author'] = df.duplicated(subset='authors', keep=False)

    # 작가 이름이 완전히 같지 않으면 제목과 출판사, 장르만 고려하기
    df['content'] = df[['bookname', 'publisher']].apply(lambda x: ' '.join(x), axis=1)
    df.loc[df['same_author'], 'content'] = df.loc[df['same_author'], 'content'] + ' ' + df.loc[df['same_author'], 'authors']


    # TfidfVectorizer 생성
    vectorizer = TfidfVectorizer(stop_words='english')

    # 모든 요소에 대해 Tf-idf vectorization 적용
    content_matrix = vectorizer.fit_transform(df['content'])

    # 코사인 유사도 구하기
    cosine_sim = cosine_similarity(content_matrix)

    

    # favorites 책들을 담을 빈 배열 indices 생성
    indices = []

    # favorites 책에서
    for isbn in favorites:
        # isbn 값과 일치하는 인덱스 가져오기
        idx = df[df['isbn13'] == isbn].index[0]
        # indices에 그 책 추가
        indices.append(idx)

    # favorites 책으로 책의 유사도 점수 목록 만들기
    sim_scores = [list(enumerate(cosine_sim[i])) for i in indices]
    sim_scores = [item for sublist in sim_scores for item in sublist]

    # 유사도 점수가 높은 수능로 정렬하기
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 유사도 높은 순에서 가장 유사도 높은 len(favorites) 제거하고 n개 가져오기
    top_n = [i[0] for i in sim_scores[len(favorites) : n + len(favorites)]]

    # 가장 유사도 높은 n개의 책 가져오기
    return df.iloc[top_n]


# 함수 호출
print(recommend_books([9788954622035], 50))

             isbn13                         bookname                  authors  \
2921  9788954610131                  검은 꽃 :김영하 장편소설                       김영하   
2676  9788954604178                     퀴즈쇼:김영하 장편소설                      김영하   
1386  9788954625470                      보다 :김영하 산문                  지은이: 김영하   
198   9788954645614                 오직 두 사람 :김영하 소설                  지은이: 김영하   
2511  9788982817144                   검은 꽃:김영하 장편 소설                   김영하 지음   
831   9788954617628            너의 목소리가 들려 :김영하 장편소설                  지은이: 김영하   
285   9788954655972                  여행의 이유 :김영하 산문                  지은이: 김영하   
4552  9788954610148                오빠가 돌아왔다 :김영하 소설                       김영하   
2993  9788954611770      나는 나를 파괴할 권리가 있다 :김영하 장편소설                  지은이: 김영하   
1633  9788954611763        무슨 일이 일어났는지는 아무도 :김영하 소설                  지은이: 김영하   
2779  9788954601917  빛의 제국=김영하 장편소설/Empire of lights                      김영하   
3940  9788936436766         